In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

poem_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a poet who write poetry about programming languages.",
        ),
        ("human", "I want a poet about {programming_language}."),
    ]
)

poem_chain = poem_prompt | chat

In [5]:
description_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are a descriptor about poetry. If you got a poem, then"
                " you explain it."
            ),
        ),
        ("human", "{poem}"),
    ]
)

description_chain = description_prompt | chat

final_chain = {"poem": poem_chain} | description_chain

final_chain.invoke({"programming_language": "Python"})

In the land of code, where serpents slither,
There lies a language, elegant and clever.
Python, the snake that charms with grace,
Guiding programmers in a seamless embrace.

Its syntax clean, like a river's flow,
Indentation matters, as all coders know.
With libraries vast, like stars in the sky,
Python shines bright, never asking why.

From web development to data science's art,
Python weaves its magic, touching every part.
A language beloved by beginners and pros,
In the world of coding, Python truly glows.

So raise your glass to this language divine,
Python, the serpent that continues to shine.
In the realm of programming, it reigns supreme,
A poet's muse, a coder's dream.This poem beautifully captures the essence of Python programming language. It describes Python as a charming and elegant language that guides programmers with grace and ease. The poem highlights Python's clean syntax, the importance of indentation, and the vast libraries available for developers to use. It also me

AIMessageChunk(content="This poem beautifully captures the essence of Python programming language. It describes Python as a charming and elegant language that guides programmers with grace and ease. The poem highlights Python's clean syntax, the importance of indentation, and the vast libraries available for developers to use. It also mentions the versatility of Python in various fields such as web development and data science, making it a favorite among both beginners and experienced programmers. The poem concludes by praising Python as a shining star in the world of coding, a language that continues to reign supreme and inspire both poets and coders alike.")